<a href="https://colab.research.google.com/github/pierce3215/finances/blob/main/LSTM_Medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
import plotly.express as px
import numpy as np

In [2]:
ticker = input("ENTER THE STOCK WHOSE PREDICTION YOU WANT : ")
df = yf.download(ticker, start='2017-01-01' , end='2023-10-06')
print(df.tail())

ENTER THE STOCK WHOSE PREDICTION YOU WANT : SPY
[*********************100%%**********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-09-29  431.670013  431.850006  425.910004  427.480011  427.480011   
2023-10-02  426.619995  428.600006  424.459991  427.309998  427.309998   
2023-10-03  425.059998  427.369995  420.179993  421.589996  421.589996   
2023-10-04  422.070007  425.429993  420.559998  424.660004  424.660004   
2023-10-05  424.359985  425.369995  421.170013  424.500000  424.500000   

               Volume  
Date                   
2023-09-29  115078500  
2023-10-02   83798600  
2023-10-03  103760600  
2023-10-04   87453000  
2023-10-05   70103300  


In [4]:
df.describe

<bound method NDFrame.describe of                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2017-01-03  225.039993  225.830002  223.880005  225.240005  200.629745   
2017-01-04  225.619995  226.750000  225.610001  226.580002  201.823257   
2017-01-05  226.270004  226.580002  225.479996  226.399994  201.662979   
2017-01-06  226.529999  227.750000  225.899994  227.210007  202.384445   
2017-01-09  226.910004  227.070007  226.419998  226.460007  201.716415   
...                ...         ...         ...         ...         ...   
2023-09-29  431.670013  431.850006  425.910004  427.480011  427.480011   
2023-10-02  426.619995  428.600006  424.459991  427.309998  427.309998   
2023-10-03  425.059998  427.369995  420.179993  421.589996  421.589996   
2023-10-04  422.070007  425.429993  420.559998  424.660004  424.660004   
2023-10-05  424.359985  425.369995  421.170013  424.500000  424.500000   

   

In [14]:
training_set = df.iloc[:1701, 1:2].values
test_set = df.iloc[1701:, 1:2].values

In [15]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [16]:
X_train = []
y_train = []
for i in range(120, 1700):
    X_train.append(training_set_scaled[i-120:i, 0])
    y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [18]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding first LSTM layer and some dropout Dropout regularisation
regressor.add(LSTM(units=100, return_sequences=True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding second LSTM layer and some dropout Dropout regularisation
regressor.add(LSTM(units=100, return_sequences=True))
regressor.add(Dropout(0.2))

# Adding third LSTM layer and some dropout Dropout regularisation
regressor.add(LSTM(units=100, return_sequences=True))
regressor.add(Dropout(0.2))

# Adding fourth LSTM layer and some dropout Dropout regularisation
regressor.add(LSTM(units=100, return_sequences=True))
regressor.add(Dropout(0.2))

# Adding fifth LSTM layer and some dropout Dropout regularisation
regressor.add(LSTM(units=100))
regressor.add(Dropout(0.2))

# Adding the Output Layer
regressor.add(Dense(units=1))

# Compiling the RNN
# Because we're doing regression hence mean_squared_error
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 10, batch_size = 16)

Epoch 1/10
99/99 [==============================] - 69s 585ms/step - loss: 0.0193
Epoch 2/10
99/99 [==============================] - 51s 511ms/step - loss: 0.0053
Epoch 3/10
17/99 [====>.........................] - ETA: 38s - loss: 0.0042

KeyboardInterrupt: ignored

In [ ]:
# Getting the real stock price of 2017
dataset_test = pd.read_csv('Aaple_Stock_Price_Test.csv')
# We want to create a numpy arrary not a vector hence 1:2
real_stock_price = dataset_test.iloc[:, 1:2].values
# Getting the predicted stock price of Jan 2020
# We have to concatenate both sets as the 60 T will have data from Dec 2019 and January 2020
# We shouldn't concatenate the train and test set as we would have to apply feature scaling
# to the concatenated result which would change the test prices
# Hence we will concatenate the dataset and then scale them
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 120:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(120, 141):
    X_test.append(inputs[i-120:i, 0])

X_test = np.array(X_test)
# 3D format
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
# Inverse the scaling
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# Visualising the results
plt.plot(real_stock_price, color='Red', label='Real Aaple Stock Price')
plt.plot(predicted_stock_price, color='Blue', label='Predicted Aaple Stock Price')
plt.title('Aaple Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Aaple Stock Price')
plt.legend()
plt.show()